In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import string
import numpy as np
import os
import glob

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [15]:
def get_page_source(url):
  r=requests.get(url,headers=headers)
  soup= BeautifulSoup(r.content, 'lxml')
  return soup

# Get POST.csv and COMMENT.csv


In [60]:
def sub_post(url,page_start,page_end):
  """
  Crawl data from link: https://vfo.vn/forum/hoi-dap-nhanh.70/

  """
  soup= get_page_source(url)
  max_page=int(soup.find('div',class_="inputGroup inputGroup--numbers inputNumber").find('input')['max'])
  while(page_start>0 and page_end < max_page+1):

    title_post=[]
    url_post=[]
    views=[]
    comments=[]
    
    page=page_start
    while(page < page_end+1):
      url_by_page=url+'page-'+str(page)
      soup=get_page_source(url_by_page)
      title_post+=[ce.find('a').text for ce in soup.findAll('div',class_='structItem-title')]
      url_post+=['https://vfo.vn/'+ce.findAll('a', href=True)[0]['href'] for ce in soup.findAll('div',class_='structItem-title')]
      views+=[int(ce.find('dd').text.replace('K','000')) for ce in soup.findAll('dl',class_='pairs pairs--justified structItem-minor')]
      comments+=[int(ce.find('dd').text.replace('K','000')) for ce in soup.findAll('dl',class_='pairs pairs--justified')]
      print(page)
      page+=1
    df= pd.DataFrame({'title_post':title_post,'url_post':url_post,'views':views,'comments':comments})  
    return df
  return 'ERROR!!!'

In [69]:
#trial with several pages: 777-779
sub_post_=sub_post('https://vfo.vn/forum/hoi-dap-nhanh.70/',777,779)

777
778
779


In [48]:

def full_post_and_comments(sub_post):
  '''
  sub_post: return value from sub_post(url,page_start,page_end) function.
  Set 'id_post' by the index for sub_post before running this function.

  '''
  post=sub_post.copy()
  url_topic_end='page-{}'
  #for extra_post
  content_post=[]
  tags=[]
  #name_author_post=[]
  id_author_post=[]
  url_author_post=[]
  datePublished_post=[]

  #for comment
  #name_author_cmt=[]
  id_author_cmt=[]
  url_author_cmt=[]
  content_cmt=[]
  datePublished_cmt=[]
  id=[]

  page=1
  for index,row in post.iterrows():
    print(index)
    url=row['url_post']
    id_post=row['id_post']
    soup=get_page_source(url+url_topic_end.format(1))
    if soup.find('ul',class_="pageNav-main") is None:
      end=2
    else:
      end=int(soup.find('ul',class_="pageNav-main").findAll('li')[-1].text)+1
    for page in range(1,end):
      post_detail= get_page_source(url+url_topic_end.format(page))
      if(page==1):
        content_post.append(post_detail.find('article',class_="message-body js-selectToQuote").text)
        if(post_detail.find('span',class_="js-tagList")==None): tags.append([])
        else: tags.append([tag.text.strip() for tag in post_detail.find('span',class_="js-tagList").findAll("a")])
        author=soup.find('div', class_="v-des")
        #name_author_post.append(author.find('a',class_="username user-thread u-concealed").text)
        id_author_post.append(int(author.find('a',class_="username user-thread u-concealed")['data-user-id']))
        url_author_post.append(author.find('a',class_="username user-thread u-concealed")['href'])
        datePublished_post.append(soup.find('li',class_="u-concealed").find("time", class_='u-dt')['datetime'])
        
      start=0
      if(page==1): start=1
      for cmt in post_detail.findAll('article',class_="message message--post js-post js-inlineModContainer ")[start:]:
        if(cmt.find('h4',class_="message-name").text=='Oda'): pass
        else:
          id.append(id_post)
          #name_author_cmt.append(cmt.find('h4',class_="message-name").find("a", class_="username ").text)
          id_author_cmt.append(int(cmt.find('h4',class_="message-name").find("a", class_="username ")['data-user-id']))
          url_author_cmt.append(cmt.find('h4',class_="message-name").find("a", class_="username ")['href'])
          content_cmt.append(cmt.find('div',class_="bbWrapper").text)
          datePublished_cmt.append(cmt.find('li',class_="u-concealed").find("time", class_='u-dt')['datetime'])
          page+=1
  
  post['content_post']=content_post
  post['tags']=tags
  #post['name_author_post']=name_author_post
  post['id_author_post']=id_author_post
  post['url_author_post']=url_author_post
  post['datePublished_post']=datePublished_post

  #comment=pd.DataFrame({'id_post':id,'name_author_cmt':name_author_cmt,'id_author_cmt':id_author_cmt,'url_author_cmt':url_author_cmt,'content_cmt':content_cmt,'datePublished_cmt':datePublished_cmt})
  comment=pd.DataFrame({'id_post':id,'id_author_cmt':id_author_cmt,'url_author_cmt':url_author_cmt,'content_cmt':content_cmt,'datePublished_cmt':datePublished_cmt})
  return  post, comment



In [70]:
#Set 'id_post' by the index for sub_post
sub_post_['id_post']=np.arange(0,len(sub_post_))
sub_post_

,title_post,url_post,views,comments,id_post
0,Máy tính bị lỗi icon trên Windows 7,https://vfo.vn//r/may-tinh-bi-loi-icon-tren-wi...,14000,108,0
1,Giúp mình giải đáp cái này với :)),https://vfo.vn//r/giu%CC%81p-mi%CC%80nh-gia%CC...,2000,2,1
2,lỗi trên store win 10,https://vfo.vn//r/loi-tren-store-win-10.79151/,2000,3,2
3,Em đang định mua 1 dây cáp HDMI để phát qua TV...,https://vfo.vn//r/em-dang-dinh-mua-1-day-cap-h...,2000,2,3
4,Lỗi game mafia 2,https://vfo.vn//r/loi-game-mafia-2.79138/,4000,7,4
5,Máy bị Not Responding khi Format,https://vfo.vn//r/may-bi-not-responding-khi-fo...,3000,8,5
6,Tư vấn xài phần mềm coi phim nào ạ?,https://vfo.vn//r/tu-van-xai-phan-mem-coi-phim...,2000,4,6
7,Lỗi [ ERR_CONNECTION_TIMED_OUT] Không vào được...,https://vfo.vn//r/loi-err_connection_timed_out...,3000,3,7
8,Tư vấn chọn trường học CNTT,https://vfo.vn//r/tu-van-chon-truong-hoc-cntt....,2000,5,8
9,Cần người cài hộ driver âm thanh,https://vfo.vn//r/can-nguoi-cai-ho-driver-am-t...,2000,6,9


In [71]:
posts_comments=full_post_and_comments(sub_post_)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [72]:
posts_comments[0]

,title_post,url_post,views,comments,id_post,content_post,tags,id_author_post,url_author_post,datePublished_post
0,Máy tính bị lỗi icon trên Windows 7,https://vfo.vn//r/may-tinh-bi-loi-icon-tren-wi...,14000,108,0,\nbọn bạn em nó mượn máy rồi không biết nó vào...,[],322324,/m/thanhdz1798.322324/,2016-01-09T21:08:53+0700
1,Giúp mình giải đáp cái này với :)),https://vfo.vn//r/giu%CC%81p-mi%CC%80nh-gia%CC...,2000,2,1,\nChỉ là do cái app Zing Mp3 của VNG lúc ...,[],300784,/m/snappy.300784/,2016-01-10T07:05:24+0700
2,lỗi trên store win 10,https://vfo.vn//r/loi-tren-store-win-10.79151/,2000,3,2,"\nnhư hình,ai giúp với.....\n\n\t\t\t{\n\t\t\t...",[],225300,/m/ngvyen.225300/,2016-01-10T00:10:26+0700
3,Em đang định mua 1 dây cáp HDMI để phát qua TV...,https://vfo.vn//r/em-dang-dinh-mua-1-day-cap-h...,2000,2,3,\nem đang định mua 1 dây cáp HDMI để phát ...,[],288492,/m/t-v.288492/,2016-01-10T08:10:07+0700
4,Lỗi game mafia 2,https://vfo.vn//r/loi-game-mafia-2.79138/,4000,7,4,"\nTôi copy từ máy chơi được về, khi chép qua m...",[],322315,/m/chanhuy2006.322315/,2016-01-09T20:01:14+0700
5,Máy bị Not Responding khi Format,https://vfo.vn//r/may-bi-not-responding-khi-fo...,3000,8,5,\nXin Chào các bạn. Máy mình có mắc một vấn đề...,[],322126,/m/le-van-thinh.322126/,2016-01-08T12:09:51+0700
6,Tư vấn xài phần mềm coi phim nào ạ?,https://vfo.vn//r/tu-van-xai-phan-mem-coi-phim...,2000,4,6,"\nlúc trước xài KM Player, sau một thời gian e...",[],317523,/m/ducvinhcdct.317523/,2016-01-09T21:54:16+0700
7,Lỗi [ ERR_CONNECTION_TIMED_OUT] Không vào được...,https://vfo.vn//r/loi-err_connection_timed_out...,3000,3,7,\nnảy đang tán gái bỗng nhiên máy em không vào...,[],273457,/m/hacvuonghcvg.273457/,2016-01-09T19:55:14+0700
8,Tư vấn chọn trường học CNTT,https://vfo.vn//r/tu-van-chon-truong-hoc-cntt....,2000,5,8,"\nnhư tiêu đề đã nói, em năm nay mới hết 12 ! ...",[],322320,/m/phamhuudo.322320/,2016-01-09T20:36:54+0700
9,Cần người cài hộ driver âm thanh,https://vfo.vn//r/can-nguoi-cai-ho-driver-am-t...,2000,6,9,\nmọi người ơi ai biết cài driver âm thanh thì...,[],281926,/m/nambigbang.281926/,2016-01-09T20:24:43+0700


In [73]:
posts_comments[1]

,id_post,id_author_cmt,url_author_cmt,content_cmt,datePublished_cmt
0,0,286410,/m/eldorado.286410/,Reply: máy bị lỗi\n\nLỗi gì bác \nbình thường ...,2016-01-09T21:11:08+0700
1,0,286410,/m/eldorado.286410/,Reply: máy bị lỗi\n\nà mấy cái icon chớ gì (kh...,2016-01-09T21:12:16+0700
2,0,322324,/m/thanhdz1798.322324/,Reply: máy bị lỗi\n\nmở nó không được bác à. n...,2016-01-09T21:13:09+0700
3,0,322324,/m/thanhdz1798.322324/,Reply: máy bị lỗi\n\nbiết giúp mình với ạ,2016-01-09T21:14:51+0700
4,0,177479,/m/suu-nguyen.177479/,Reply: máy bị lỗi\n\nKhông phải lỗi gì cả bạn ...,2016-01-09T21:15:00+0700
...,...,...,...,...,...
495,59,286410,/m/eldorado.286410/,"\n\nKin Trần:\n\n\n\n\t\t\tTeamview là gì dọ, ...",2016-01-06T13:22:02+0700
496,59,321331,/m/kin-tran.321331/,"Bác thông minh vãi, cho biết nick fb đi ạ",2016-01-06T14:05:09+0700
497,59,321331,/m/kin-tran.321331/,"Bây giờ không tiên lắm ạ, bác cho em biết nick...",2016-01-06T14:07:51+0700
498,59,286410,/m/eldorado.286410/,"bạn muốn lấy nick vsupport, latruongthinh hay ...",2016-01-06T19:42:42+0700


# Get USER.csv

In [54]:

def get_users(df):
  """
  df: user_list dataframe .
  result: dataframe contains user information.
  not_available: list contains URL of inactive users.
  """
  id_user=[]
  name_user=[]
  star=[]
  DateJoin=[]
  DateLastAccess=[]
  info=[]

  not_available=[]
  for i,row in df.iterrows():
    print(i)
    user=get_page_source('https://vfo.vn{}#about'.format(row['url_user']))
    if(user.find('div',class_="blockMessage blockMessage--error blockMessage--iconic")==None):
      name_user.append(user.find('div',class_='memberHeader-content memberHeader-content--info').find('span',class_="username ").text)
      id_user.append(user.find('div',class_='memberHeader-content memberHeader-content--info').find('span',class_="username ")['data-user-id'])
      star_source=user.find("span",class_="userTitle",dir="auto")
      if(star_source==None  or '✩' not in star_source.text): star.append(None)
      else: star.append(len(star_source))
      DateJoin.append(user.findAll('dl', class_="pairs pairs--inline")[0].find('time')['datetime'])
      if(len(user.findAll('dl', class_="pairs pairs--inline"))==1): DateLastAccess.append(None)
      else: DateLastAccess.append(user.findAll('dl', class_="pairs pairs--inline")[1].find('time')['datetime'])
      info.append([x.replace(r',','') for x in user.find('div',class_="pairJustifier").text.split() if x.replace(r',','').isdigit()])
    else: 
      not_available.append(row['url_user'])
      pass
  result=pd.DataFrame({'id_user':id_user,'name_user':name_user,'star':star,'DateJoin':DateJoin,'DateLastAccess':DateLastAccess,'info':info})
  result=pd.concat([result,pd.DataFrame(result['info'].to_list(), columns=['comment_count','Reaction_Score','Score'])], axis=1).drop('info',axis=1)
  return result,not_available
  

In [74]:
POST=posts_comments[0]
COMMENT=posts_comments[1]

In [75]:
# create user_list
user_list=pd.DataFrame(pd.concat([POST.url_author_post,COMMENT.url_author_cmt]), columns=['url_user']).drop_duplicates(ignore_index=True)
user_list

,url_user
0,/m/thanhdz1798.322324/
1,/m/snappy.300784/
2,/m/ngvyen.225300/
3,/m/t-v.288492/
4,/m/chanhuy2006.322315/
...,...
96,/m/hoangdiephbg.292448/
97,/m/viet-anh-it.271951/
98,/m/apink94.301743/
99,/m/thiennamlong.205751/


In [76]:
get_users(user_list)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


(   id_user     name_user  star  ... comment_count Reaction_Score Score
 0   300784        SNAPPY   NaN  ...            57              0     0
 1   225300        ngvyen   NaN  ...           128              0     0
 2   288492           t-v   NaN  ...           185              0     0
 3   322126  Lê Văn Thịnh   NaN  ...             4              0     0
 4   317523   ducvinhcdct   NaN  ...            40              0     0
 ..     ...           ...   ...  ...           ...            ...   ...
 84  309230     haibongbi   NaN  ...           172              0     0
 85  292448  hoangdiephbg   1.0  ...           128              0   166
 86  271951   Việt Anh IT   NaN  ...           627              0     0
 87  301743       apink94   NaN  ...            14              0     0
 88  274611    truongtvat   1.0  ...            27              0    16
 
 [89 rows x 8 columns],
 ['/m/thanhdz1798.322324/',
  '/m/chanhuy2006.322315/',
  '/m/phamhuudo.322320/',
  '/m/minhtu123456.322068/',